<a href="https://colab.research.google.com/github/flagarh/Assistente-de-Interpreta-o-M-dica_Imers-o-em-IA-da-Alura/blob/main/_Assistente_de_int%C3%A9rprete_m%C3%A9dico_AI_(EN_ES)_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################


# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    # Inserir a tool de busca do Google ###############################################
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
##########################################
# --- Agente 1: Agente Traductor --- #
##########################################
def agente_buscador(topico):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.0-flash",
        instruction = """
        Eres un asistente de interpretacion medica. Tu tarea es traducir palabras y frases usando la herramienta de busqueda de google (google_search).
        Realizas interpretaciones en Español e Ingles en el contexto medico.
        Utiliza la terminologia medica correcta en ambos idiomas
        Mantén el estilo y tono del texto original en la traducción.
        cunado escriba una palabra me proporcionaras su traduccion al ingles o al español, asegurandote de que la traduccion sea precisa dentro del contexto de la interpretacion medica
        siempre las respuestas seran en el siguiente formato
        Ejemplos de respuesta
        Entrada: Aneurysm
        Salida esperada: Aneurysm / Aneurísma
        ----------------------
        Entrada: Paro cardíaco
        Salida esperada: Paro cardíaco / Cardiac arrest
        ----------------------
        Entrada: Palsy
        Salida esperada: Cerebral Palsy / Parálisis cerebral
        ----------------------
        Entrada: Painkiller/ pain reliever
        Salida esperada: Painkiller/ pain reliever / Analgésico
        """ ,
        description="Agente que busca traducciones de palabras en Google",
        tools=[google_search]
    )

    # These lines were moved inside the function definition
    entrada_do_agente_buscador = f"Tópico: {topico}"
    # Executa o agente
    resposta = call_agent(buscador, entrada_do_agente_buscador)
    return resposta

In [ ]:
################################################
# --- Agente 2: Sugiere Vocabulario --- #
################################################
def agente_planejador(topico_2):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        # Inserir as instruções do Agente Planejador #################################################
        instruction="""
        Eres un asistente de interpretacion medica. Tu tarea es sugerir vocabulario relacionado con una tematica en especifico.
        El vocabulario sugerido muestra una lista de minimo 10 palabras y hasta 20 palabras traducidas en ingles y espanol que esten directamente relacionas con la tematica solicitada.
        La busqueda se hace con herramienta de busqueda de google (google_search).
        Siempre reponderas en el siguiente formato
        Ejemplo de una lista de vocabulario de una tematica:
        Entrada: Diabetes
        Salida esperada: Hello! We can provide an interpretation for diabetes.
        Here's a list of common diabetes-related terms in English and Spanish:
        Diabetes - Diabetes
        Blood sugar - Azúcar en la sangre / Glucosa en la sangre
        Insulin - Insulina
        Hyperglycemia - Hiperglucemia
        Hypoglycemia - Hipoglucemia
        A1C test - Prueba de A1C / Prueba de hemoglobina glicosilada
        Type 1 diabetes - Diabetes tipo 1
        Type 2 diabetes - Diabetes tipo 2
        Gestational diabetes - Diabetes gestacional
        Glucose meter - Glucómetro
        Insulin injection - Inyección de insulina
        Insulin pump - Bomba de insulina
        Oral medication - Medicamento oral
        Diet - Dieta
        Exercise - Ejercicio
        Neuropathy - Neuropatía
        Retinopathy - Retinopatía
        Nephropathy - Nefropatía
        --------------------------------------------------------
        Entrada: Female Reproductive System
        Salida esperada: Hello! We can provide an interpretation for female reproductive system.
        Here's a list of common female reproductive system-related terms in English and Spanish:
        Female Reproductive system - Sistema reproductivo femenino
        birth canal - canal de parto
        cervix - cuello uterino
        clitoris - clitoris
        eggs - ovulos
        endometrium - endometrio
        fallopian tubes - trompas de falopio
        fundus - fondo del utero
        ovaries - ovarios
        uterine lining - revestimiento uterino
        uterus - utero
        vagina - vagina
        womb - vientre / claustro / matriz
        placenta - placenta

        """,
        description="Agente que busca traducciones de palabras en Google y sugiere vocabulario relacionado",
        tools=[google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{topico_2}"
    # Executa o agente
    plano_do_post = call_agent(planejador, entrada_do_agente_planejador)
    return plano_do_post

In [ ]:

print("Welcome to the medical interpretation support system")

# --- Obter o Tópico do Usuário ---
topico = input("❓ What word do you want me to translate?: ")


# Inserir lógica do sistema de agentes ################################################
if not topico:
    print("You forgot to type the topic!")
else:
    print(f"Wonderful! Let's translate then {topico}")
    lancamentos_buscados = agente_buscador(topico)
    print("\n--- 📝 Resultado do Agente 1 (Traductor) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")

# Corrected indentation for the while loop and its content
topico = input("❓ What word do you want me to translate?: ")
while topico != "fin":
    print(f"Wonderful! Let's translate then {topico}")
    lancamentos_buscados = agente_buscador(topico)
    print("\n--- 📝 Resultado do Agente 1 (Traductor) ---\n")
    display(to_markdown(lancamentos_buscados))
    print("--------------------------------------------------------------")
    topico = input("❓ What word do you want me to translate?: ") # Added input inside the loop to allow breaking out

Welcome to the medical interpretation support system
❓ What word do you want me to translate?: eye
Wonderful! Let's translate then eye

--- 📝 Resultado do Agente 1 (Traductor) ---



> Tópico: eye / Tópico: ojo
> 


--------------------------------------------------------------
❓ What word do you want me to translate?: leg
Wonderful! Let's translate then leg

--- 📝 Resultado do Agente 1 (Traductor) ---



> Leg / Pierna
> 


--------------------------------------------------------------
❓ What word do you want me to translate?: fin


In [ ]:
print("Welcome to the medical interpretation support system")

topico_2 = input("❓ What vocabulary do you want me to suggest to you?: ")

# Inserir lógica do sistema de agentes ################################################
if not topico_2:
    print("You forgot to type the topic!")
else:
    print(f"Wonderful! Let's translate and find vocabulary related to {topico_2}")
    plano_de_post = agente_planejador(topico_2)
    print("\n--- 📝 Resultado do Agente 2 (Sugiere Vocabulario) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")
topico_2 = input("❓ What vocabulary do you want me to suggest to you?: ")
while topico_2 != "fin":
    print(f"Wonderful! Let's translate and find vocabulary related to {topico_2}")
    plano_de_post = agente_planejador(topico_2)
    plano_de_post = agente_planejador(topico_2)
    print("\n--- 📝 Resultado do Agente 2 (Sugiere Vocabulario) ---\n")
    display(to_markdown(plano_de_post))
    print("--------------------------------------------------------------")
    topico_2 = input("❓ What vocabulary do you want me to suggest to you?: ")


❓ What vocabulary do you want me to suggest to you?: fin
Wonderful! Let's translate and find vocabulary related to fin

--- 📝 Resultado do Agente 2 (Sugiere Vocabulario) ---



> ¡Hola! Podemos proporcionar una interpretación para el tema "Fin".
> Aquí hay una lista de términos comunes relacionados con "Fin" en inglés y español:
> 
> *   End - Fin
> *   Termination - Terminación
> *   Aim - Objetivo
> *   Purpose - Propósito
> *   Objective - Objetivo
> *   Finance - Finanzas
> *   Financial - Financiero
> *   Money - Dinero
> *   Financial costs - Costos financieros
> *   Financial assets - Activos financieros
> *   Financial forecasting - Pronóstico financiero
> *   Balance - Balance
> *   Bank - Banco
> *   Loan - Préstamo
> *   Interest - Interés
> *   Mortgage - Hipoteca
> *   Investment - Inversión
> *   Currency - Divisa
> *   Savings account - Cuenta de ahorros
> *   Account - Cuenta


--------------------------------------------------------------
❓ What vocabulary do you want me to suggest to you?: fin
